# Setup & Training Script
To be run after new videos are added

Based on the article [Guide to Build Video Classification Model](https://www.analyticsvidhya.com/blog/2019/09/step-by-step-deep-learning-tutorial-video-classification-python/)


In [1]:
!pip3 install opencv-python

### Importing Libraries

In [1]:
import os, os.path
import cv2  # for caputring videos
import math # for mathematical operations
import matplotlib.pyplot as plt # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image # for preprocessing the images
import numpy as np # for mathematical operations
from keras.utils import np_utils
from numpy import genfromtxt
from skimage.transform import resize # for resizing images
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm

Using TensorFlow backend.


### Storing the name of videos in a dataframe
First bit opens the file, reads the file, and splits them by "enter"

Second bit creates a dataframe and stores the above extracted video names under "train['video_name']"

In [2]:
# open the .txt file which have names of training videos
# f = open("trainlist_tennis.txt", "r")
# temp = f.read()
# videos = temp.split("\n")
# print(videos)

# creating a dataframe having video names
# train = pd.DataFrame()
# train['video_name'] = videos
# train

 # Turn the csv files into dictionaries
1. Opens and converts csv file
2. Gets tags
3. Gets points for each frame
4. Reshapes each frame array
5. Groups frames by three

In [21]:
path="dataPoints_training/"

convertFile = pd.DataFrame()

for filename in glob(os.path.join(path, '*.csv')):    
    file = filename.split("\\")[1]
    convertFile = pd.read_csv(path + file, header=None)
    locals()['trainingFrames{}'.format("_" + file.split(".csv")[0])] = {}
    locals()['trainingCombo{}'.format("_" + file.split(".csv")[0])] = {}
    
    tag = filename.split("\\")[1].split("_")[3].split(".csv")[0]
    locals()['trainingCombo{}'.format("_" + file.split(".csv")[0])]["tag"] = tag            
        
    size = len(convertFile[0]) // 12
       
    n = 12
    for i in range((len(convertFile) + 12 - 1) // 12):
        data = convertFile.to_numpy()[i * n:(i + 1) * n]
        data = np.delete(data, 0, 1)
        data = np.reshape(data, (24, 1))
        locals()['trainingFrames{}'.format("_" + file.split(".csv")[0])]['{}'.format(i)] = data

    for i in locals()['trainingFrames{}'.format("_" + file.split(".csv")[0])]:                
        if int(i) < (len(locals()['trainingFrames{}'.format("_" + file.split(".csv")[0])]))-3:
            j = int(i)+1
            k = int(i)+2
            
            combine = locals()['trainingFrames{}'.format("_" + file.split(".csv")[0])]['{}'.format(i)]
            combineTwo = locals()['trainingFrames{}'.format("_" + file.split(".csv")[0])]['{}'.format(j)]
            combineThree = locals()['trainingFrames{}'.format("_" + file.split(".csv")[0])]['{}'.format(k)]
                        
            combine = np.append(combine, combineTwo, axis=1)
            combine = np.append(combine, combineThree, axis=1)
            
            locals()['trainingCombo{}'.format("_" + file.split(".csv")[0])]['{}'.format(i)] = combine            

### Create a similar dataframe for the test videos
Does the same thing as above but for the test video names

In [8]:
# # open the .txt file which have names of test videos
# f = open("testlist_tennis.txt", "r")
# temp = f.read()
# videos = temp.split('\n')

# # creating a dataframe having video names
# test = pd.DataFrame()
# test['video_name'] = videos
# test

# Do the same for test videos

In [ ]:
path="dataPoints_test/"

convertFile = pd.DataFrame()

for filename in glob(os.path.join(path, '*.csv')):    
    file = filename.split("\\")[1]
    convertFile = pd.read_csv(path + file, header=None)
    locals()['testFrames{}'.format("_" + file.split(".csv")[0])] = {}
    locals()['testCombo{}'.format("_" + file.split(".csv")[0])] = {}
        
    size = len(convertFile[0]) // 12
    
    for i in range((len(convertFile) + 12 - 1) // 12):
        data = convertFile.to_numpy()[i * n:(i + 1) * n]
        data = np.delete(data, 0, 1)
        data = np.reshape(data, (24, 1))
        locals()['testFrames{}'.format("_" + file.split(".csv")[0])]['{}'.format(i)] = data

    for i in locals()['testFrames{}'.format("_" + file.split(".csv")[0])]:                
        if int(i) < len(locals()['testFrames{}'.format("_" + file.split(".csv")[0])])-3:
            j = int(i)+1
            k = int(i)+2
            
            combine = locals()['testFrames{}'.format("_" + file.split(".csv")[0])]['{}'.format(i)]
            combineTwo = locals()['testFrames{}'.format("_" + file.split(".csv")[0])]['{}'.format(j)]
            combineThree = locals()['testFrames{}'.format("_" + file.split(".csv")[0])]['{}'.format(k)]
            
            combine = np.append(combine, combineTwo, axis=1)
            combine = np.append(combine, combineThree, axis=1)
            
            locals()['testCombo{}'.format("_" + file.split(".csv")[0])]['{}'.format(i)] = combine            

### Adding the tags
For both training and test sets: grabs the tag (0 or 1/good or bad serve) from the text file and stores in the respective dataframe.

In [5]:
# # creating tags for training videos
# train_video_tag = []
# for i in range(train.shape[0]):
#     train_video_tag.append(train['video_name'][i].split('_')[1].split('.')[0])
    
# train['tag'] = train_video_tag

# # creating tags for test videos
# test_video_tag = []
# for i in range(test.shape[0]):
#     test_video_tag.append(test['video_name'][i].split('_')[1].split('.')[0])
    
# test['tag'] = test_video_tag

In [9]:
#This is what the test dataframe looks like 
# test

In [10]:
# This is what the train dataframe looks like 
# train

### Frame Extraction and Storing
All frames are put in a folder named train_1

In [11]:
# # open the .txt file which have names of training videos
# f = open("trainlist_tennis.txt", "r")
# temp = f.read()
# videos = temp.split("\n")

# train = pd.DataFrame()
# train['video_name'] = videos

# # storing the frames from training videos
# for i in tqdm(range(len(file))):
#     count = 0
#     videoFile = train['video_name'][i]
#     cap = cv2.VideoCapture('myVids/'+videoFile)   # capturing the video from the given path
#     frameRate = cap.get(5) #frame rate
#     x=1
#     while(cap.isOpened()):
#         frameId = cap.get(1) #current frame number
#         ret, frame = cap.read()
#         if (ret != True):
#             break
#         if (frameId % math.floor(frameRate) == 0):
#             # storing the frames in a new folder named train_1
#             print("this is the split")
#             print(videoFile.split(" ")[0]) #since my videos aren't in separate folder, no need to split based on ("/")
            
#             #need to make sure that the train_1 has been created
#             filename ='train_1/' + videoFile.split(' ')[0] +"_frame%d.jpg" % count;count+=1
#             cv2.imwrite(filename, frame)
#     cap.release()

### .csv file
Get the names of all the frames with their corresponding tag and put the info into a .csv file

In [9]:
# #getting the names of all the images
# images = glob("train_1/*.jpg")
# train_image = []
# train_class = []

# for i in tqdm(range(len(images))):
#     # creating the image name
#     train_image.append(images[i].split("\\")[1])  #uncomment this for windows
#     #train_image.append(images[i].split("/")[1])  #uncomment this for mac
#     # creating the class of image
#     train_class.append(images[i].split("\\")[1].split(".")[0].split("_")[1]) #uncomment this for windows
#     #train_class.append(images[i].split("/")[1].split(".")[0].split("_")[1]) #uncomment this for mac
    
# # storing the images and their class in a dataframe
# train_data = pd.DataFrame()
# train_data['image'] = train_image
# train_data['class'] = train_class

# # converting the dataframe into csv file 
# train_data.to_csv('myVids/train_new.csv',header=True, index=False)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 37/37 [00:00<?, ?it/s]


# Put all the training frames into a dummy thicc array

In [41]:
path="dataPoints_training/"

points = []

for filename in glob(os.path.join(path, '*.csv')):    
    file = filename.split("\\")[1]
    
    size = len(locals()['trainingCombo{}'.format("_" + file.split(".csv")[0])])
    
    for i in range(0, size-1):
        val = list(locals()['trainingCombo{}'.format("_" + file.split(".csv")[0])]["{}".format(i)])
        points.append(val)

# print(points[0])
X = np.array(points)
X.shape

(209, 24, 3)